### 1. Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір). Вирішіть завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.

In [1]:
import pathlib
import random
import string
import re
import io

import numpy as np
import pandas as pd

from tensorflow import data as tf_data
from tensorflow import strings as tf_strings

import keras
from keras import ops
from keras import layers
from keras.layers import TextVectorization


2024-12-19 13:05:08.617916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734606308.637516  174326 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734606308.643246  174326 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 13:05:08.664931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
text_file = pathlib.Path("out_text.txt")
print(text_file)

out_text.txt


In [5]:
path = text_file
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Corpus length: 1742828
Total chars: 39
Number of sequences: 580930


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


2024-12-19 13:05:18.913276: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [8]:
epochs = 40
batch_size = 128

x = x[:300000]
y = y[:300000]

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")


2024-12-19 13:05:19.109232: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 468000000 exceeds 10% of free system memory.


2344/2344 ━━━━━━━━━━━━━━━━━━━━ 109s 46ms/step - loss: 2.4644

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "ві хлопці оточили його так тісно, що вир"
...Generated:  аз погляду вона подіватися на собою.  за немою про собою,  відповів про на сонці на синому поспитали посереду на стого подивалися на стільки подумалися на сонцім повернув подімати помітив на сумного подиватися на стого на світів подомого постути подоволи до не подивалися на стого на стільки про не за стому поглядали подіватися на нешого подумав про це подіматися на світів постутив про те до не за 
-
...Diversity: 0.5
...Generating with seed: "ві хлопці оточили його так тісно, що вир"
...Generated:  осливо повітилися на промовив потрешти про водо ви поступалися не відомого від і знішого до рушення з аленьоменного бачення покладали побачили промов прите за не подиганому від не мого на зами, а дигорі відвовив помітно не садалися по оберхався подімнати, та зараз розмову від притечка пані собою, на д

2024-12-19 13:09:01.091310: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 468000000 exceeds 10% of free system memory.


2344/2344 ━━━━━━━━━━━━━━━━━━━━ 103s 44ms/step - loss: 1.8735

Generating text after epoch: 1
...Diversity: 0.2
...Generating with seed: "неприємна справа. адже з кожною миттю ст"
...Generated:  али посподіватися не знаєте,  за свого поспіти вона поспородив поллі.  а потім він повернутися по не був до стали за ним стали поснути до нього послухали до не посувати на свого повернути за свого по посутні послухати по столі поснути повернути посторонів пітер.  а повернути до короля, що повернутися на свого послухали потому поснутися на споку послухали на споку послухали по тому послухати потому
-
...Diversity: 0.5
...Generating with seed: "неприємна справа. адже з кожною миттю ст"
...Generated:  ав притів прийше до сибою про траві до нема сказав ти ще багато не був вигуку поморти порожив сіне,  промовив свого не зналося за стояв до справа за стали на дядько вельченні першов за свого побачити, повернути подомовли спостання бобрими за ними, вона про тому про стали в діти.  далі не загорормо кру

2024-12-19 13:13:02.704367: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 468000000 exceeds 10% of free system memory.


2344/2344 ━━━━━━━━━━━━━━━━━━━━ 113s 48ms/step - loss: 1.7494

Generating text after epoch: 2
...Diversity: 0.2
...Generating with seed: " правда ж? хіба не можна якось змінити ц"
...Generated:  е та під нами за настільним потім та відповів про того, а то так не завжди постала відповів про того,  відповів він.  не забрати старого перед ним і ви нас завжено потім випровинилися і ви нам сліб на столу відповів він про того,  відповів він,  за старову не забачили та просто відповів про того, що так не завжди поглядав потреба на столом не забалько повернувся поллі за стола.  так ви не забуло н
-
...Diversity: 0.5
...Generating with seed: " правда ж? хіба не можна якось змінити ц"
...Generated:  ілком відповів кроміння, як городно потільки забри там часом відповів він,  погодів схова. та він продовжував полова і присила. він завжди на тонкові висливою підмовоті за кровою промовили, що завжено під нас король на стрільки навіть далі до горобів, аби вони само ви стала відповів дигорі.  вона на п

2024-12-19 13:17:07.502092: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 468000000 exceeds 10% of free system memory.


2344/2344 ━━━━━━━━━━━━━━━━━━━━ 97s 41ms/step - loss: 1.6951

Generating text after epoch: 3
...Diversity: 0.2
...Generating with seed: "покінчать з нарнією, вони, напевне, посу"
...Generated:  діла на відчакі повернувся на полісту та відчувалися на поллі.  а тоді просто не витримав з них на під ного, а навіть відчували за моїми повернувся до невам на собі повернувся до нього було на пітера.  повітрі вирозував до себе слухай і завіть повів на них повернувся до нього,  повернувся на них повів до нього,  відповів дигорі.  навіть не завдилася вигук, що вони викодився на них вигук, що так во
-
...Diversity: 0.5
...Generating with seed: "покінчать з нарнією, вони, напевне, посу"
...Generated:  пів мене на світів хлопчина.  на них розповідали до нього своїми повітра. дигорі, звірів чаклунка для собі зараз у ним та під ними і поміж не виконав у там короля,  мовив дигорі.  на нього ви набігніше,  вигукнув дигорі.  а потін відчував до своїх вони на морі і повітря і вас повернувся і порожні, як м

2024-12-19 13:20:46.759044: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 468000000 exceeds 10% of free system memory.


2344/2344 ━━━━━━━━━━━━━━━━━━━━ 103s 44ms/step - loss: 1.6598

Generating text after epoch: 4
...Diversity: 0.2
...Generating with seed: " подив, що його остраханський супротивни"
...Generated:  й повне струшнув він на собі повне було потрапила в морі почали почув від нас потрібного світлом і порадив повернутися на нього.  нарешті подавалися потрапила і замовляти повернутися так він на нас про так почувся потреба,  подумав пітер.  на відкриті потрапила потреба, а то було було було потреба подумати у потрібного своїм світлом і запала в потрібний королева він про так потрапила та повне було
-
...Diversity: 0.5
...Generating with seed: " подив, що його остраханський супротивни"
...Generated:  й потямити розкриватися до нього. після він були до нього крізь кроки, і не було були звір і попередний нас говоритися його одне про неволові.  по них ви була за перед нам не можна не загородить на це поставити у крови з оцей горомними залишити так поллі на нього. ви то йому вони була за тросили вони 